In [1]:
############********** Project Completion Predictive Model **********############


#This file takes advantage of two pretrained models to perform certain tasks:

    #Model1 predicts whether a project met SLA.
    #Model2 predicts how long it takes for a project to be completed.


############*********************************************************############


In [1]:
###--- Importing the necessary libraries ---###

import pyodbc
import pickle
import pandas as pd
import numpy as np
import sys

from sklearn_pandas import CategoricalImputer
from sklearn.preprocessing import OrdinalEncoder
import holidays
from datetime import date

pd.set_option("display.max_columns", None)


In [2]:
###--- Connecting to the server to access the database ---###

conn = pyodbc.connect('Driver={SQL Server};'
'Server=HCV171DTMBPDA01\DTMBSQL16PRD.NGDS.state.mi.us,1910;'
'Database=DTMB_CentralProcurement;'
'Trusted_Connection=yes;')

cursor = conn.cursor()


In [31]:
###--- Loading the pretrained models ---###

filename = 'model1try1.sav'
filename2 = 'model2try1.sav'

loaded_model = pickle.load(open(filename, 'rb'))
loaded_model2 = pickle.load(open(filename2, 'rb'))


In [32]:
###--- Retrieving tables from the database using queries ---###

query_1 = "SELECT * FROM tblProjects WHERE ProjectType = 'RFP' Or ProjectType = 'C-RFP' Or ProjectType = 'CN' Or ProjectType = 'DS' Or ProjectType = 'CO-OP'Or ProjectType = 'ITN'Or ProjectType = 'CPC' Or ProjectType = 'RFS '"
query_2 = "SELECT * FROM tblContractCategories"


df_projects = pd.read_sql_query(query_1, conn)
df_projects = df_projects[df_projects['CategoryID'].notna()]
df_projects = df_projects.astype({'CategoryID': 'int64'})

df_level = pd.read_sql_query(query_2, conn)
df_level = df_level[['LevelCalc', 'CategoryID']]

df = df_projects.join(df_level.set_index('CategoryID'), on='CategoryID')
df = df[(df["ProjectStage"] == 'Planned') | (df["ProjectStage"] == 'Active')]

df.shape


(523, 35)

In [33]:
###--- Data pre-processing and cleaning for models---###

def prepare_df1(df):
    
    #Calculating the duration
    df = df.reset_index()
    df = df.dropna(subset=['PlanFinishDate'])
    mi_holidays = holidays.UnitedStates(state='MI')
    start = df['PlanStartDate'].astype(str)
    end = str(date.today())
    bus = []
    holi = []
    dur = []
    start_list = start.tolist()
    end_list = [end] * int(len(start_list))
    for i in range(len(start_list)):
        holi.append(len(mi_holidays[start_list[i]: end_list[i]]))
        bus.append(np.busday_count(start_list[i], end_list[i]))

    zip_object = zip(bus, holi)
    for list1_i, list2_i in zip_object:
        dur.append(list1_i-list2_i)
    df['dur'] = dur
    variables_to_remove = ['index','SharePointID', 'ID', 'ProjectName', 'CreateDate', 'PlanStartDate',
             'PlanFinishDate', 'StatusDate', 'ProjectDescription', 'Status', 'StatusDate', 'ProjectDescription',
             'Status', 'RefrenceNumber', 'ProjectStage', 'CancelationReason', 'ActualFinishDate', 'Unit',
             'SolicitationStatus', 'LastUpdatedBy', 'LastUpdateDate', 'LastUpdatedField', 'SolicitationNum',
             'SensitivityIndicator', 'SourcingCategory', 'DollarAmount']

    df = df.drop(variables_to_remove, axis = 1)
    df = df.rename(columns={"dur": "Duration"})
    df = df[df["Duration"]>0]
    # handling NaN values
    imputer = CategoricalImputer()
    data = np.array(df['Planned'], dtype=object)
    data2 = np.array(df['CreatedBy'], dtype=object)
    data3 = np.array(df['BuyerName'], dtype=object)

    df['Planned'] = imputer.fit_transform(data)
    df['CreatedBy'] = imputer.fit_transform(data2)
    df['BuyerName'] = imputer.fit_transform(data3)
    
    return df


model1_df = prepare_df1(df)
model1_df.head()


,ProjectColor,ProjectType,CreatedBy,WorkPerformedFor,BuyerName,Hours,Division,SLA,Planned,CategoryID,FedReview,ITComponent,LevelCalc,Duration
0,None,CN,calahanh,001,GP1,0.0,IT,63,No,138,False,False,4,911
1,Green,CN,calahanh,271,MJ7,12.0,IT,63,Yes,250,True,True,5,515
2,Green,CN,calahanh,271,MJ7,0.0,IT,50,Yes,250,False,True,5,33
4,Green,CN,calahanh,491,MJ7,0.0,IT,50,Yes,241,True,False,2,97
5,Green,RFP,calahanh,171,MW,0.0,IT,138,Yes,262,False,False,3,329


In [13]:
###--- Function that prepares data for model--> turns data entries to a form the model can read ---###

def prepare_inputs(X_train):
    
    oe = OrdinalEncoder(handle_unknown = "use_encoded_value", unknown_value= 569)
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    #X_test_enc = oe.transform(X_test)
    return X_train_enc


In [14]:
sla = []
for index, row in model1_df.iterrows():
    i = 1
    if row['Duration'] > row['SLA']:
        i = 0
    else:
        i = 999
    sla.append(i)

sla_arr = np.asarray(sla)
sla_ind = (np.where(sla_arr == 0)[0]).tolist()
sla_ind

[0,
 1,
 4,
 5,
 6,
 7,
 8,
 14,
 15,
 16,
 17,
 23,
 26,
 36,
 37,
 38,
 39,
 41,
 45,
 62,
 63,
 82,
 83,
 84,
 85,
 98,
 99,
 100,
 105,
 106,
 108,
 122,
 141,
 185,
 189,
 190,
 191,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 230,
 231,
 232,
 247,
 249,
 252,
 253,
 260,
 263,
 273,
 277,
 279,
 286,
 448,
 449]

In [15]:
###--- Prepare input data for the predictive model1 by selecting the best features to use ---###

X = model1_df
X = X.astype(str)
X_train = X
X_train_enc = prepare_inputs(X_train)
X_train_enc = np.delete(X_train_enc, np.s_[2, 6, 10, 11, 13], 1) #Note: Selected features may change in the fututre


In [16]:
###--- Passing data through model to get Met_SLA predictions ---###

predictions = loaded_model.predict(X_train_enc)
prob = loaded_model.predict_proba(X_train_enc)


In [17]:
pred1 = predictions.tolist()
prob1 = prob.tolist()

for ind in sla_ind:
    pred1[ind] = 0
    prob1[ind] = [1, 0]


In [18]:
###--- Calculating met_sla condition ---###



model2_df = model1_df.copy()
model2_df['Met_SLA'] = pred1
model2_df.head()


,ProjectColor,ProjectType,CreatedBy,WorkPerformedFor,BuyerName,Hours,Division,SLA,Planned,CategoryID,FedReview,ITComponent,LevelCalc,Duration,Met_SLA
0,None,CN,calahanh,001,GP1,0.0,IT,63,Yes,138,False,False,4,911,0
1,Green,CN,calahanh,271,MJ7,12.0,IT,63,Yes,250,True,True,5,515,0
2,Green,CN,calahanh,271,MJ7,0.0,IT,50,Yes,250,False,True,5,33,1
3,Green,CN,calahanh,271,MJ7,0.0,IT,50,Yes,250,False,False,5,-128,1
4,Green,CN,calahanh,491,MJ7,0.0,IT,50,Yes,241,True,False,2,97,0


In [19]:
###--- Prepare input data for the predictive model2 by selecting the best features to use ---###

X = model2_df[['ProjectColor', 'ProjectType', 'CreatedBy',  'WorkPerformedFor', 'BuyerName', 'Hours', 'Division', 'SLA', 'Planned',
         'CategoryID', 'FedReview', 'ITComponent', 'LevelCalc', 'Met_SLA']]
X = X.astype(str)
X_train_enc = prepare_inputs(X)
X_train_enc = np.delete(X_train_enc, np.s_[6, 12], 1) #Note: Selected features may change in the fututre


In [25]:
###--- Passing data through model to get Completion Period predictions ---###

predictions2 = loaded_model2.predict(X_train_enc)
prob2 = loaded_model2.predict_proba(X_train_enc)


In [26]:
met = pred1
pro = prob1
met2 = predictions2.tolist()
pro2 = prob2.tolist()
met2 = (pd.cut(np.array(met2),
       9, labels=["0-50", "51-100", "101-150", "151-200", "201-250", "251-300", "301-350", "351-400", ">400"])).tolist()


In [27]:
###--- Adding the new prediction columns to the dataframe ---###

df = df.dropna(subset=['PlanFinishDate'])

df['Met_SLA_Prediction'] = met
df['Met_SLA_PredictionProbab'] = pro
df['CompletionPeriodPrediction'] = met2
df['CompletionPeriodPredictionProbab'] = pro2


In [28]:
###--- Writing the dataframe into an output file ---###

df.to_csv('project1_predictions_edit.csv',index=False)
